In [40]:
import pandas as pd
mot_a=pd.read_csv('/kaggle/input/motors/Mot_A_rel_2.csv')
mot_b=pd.read_csv('/kaggle/input/motors/Mot_B_rel_1.csv')
mot_a

,d_alpha,h_c,r,w_t,l_t,w_o,dxIB,gamma,d_alpha_deg,hc_mm,w_o_mm,T,TR,m_Cu,m_mag,cos_phi,VM,Temp
0,0.683555,0.652960,76.388367,4.876673,15.980657,0.252080,3.958199,47.979187,20.5065,13.333585,2.261068,322.034019,46.460214,3.496543,1.637520,0.600832,449.306137,161.493855
1,0.839970,0.316877,66.528144,5.477306,21.227689,0.125430,-0.907076,38.605065,25.1991,2.806290,0.981157,319.186347,26.164250,3.422449,0.827423,0.460323,462.037013,163.451036
2,0.749656,0.430152,70.092293,5.902179,17.496837,0.198953,2.421620,32.137510,22.4898,6.407833,1.638763,365.429403,18.859974,2.550712,1.383712,0.601910,376.266906,175.349022
3,0.779947,0.588121,64.097056,3.983202,19.711625,0.325027,-2.712820,52.763373,23.3985,6.980233,2.450559,388.445550,43.657137,4.056152,1.559041,0.687662,362.526070,161.401494
4,0.671491,0.359245,61.665401,6.232320,21.751120,0.294789,1.796389,57.935310,20.1447,6.039823,2.139157,316.498500,43.529585,2.219354,0.930888,0.656835,265.543625,183.992897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,0.795862,0.389833,61.630750,4.726709,21.116910,0.391967,-0.356609,38.449031,23.8758,4.098186,2.842765,330.704739,52.500055,3.502802,0.961655,0.525826,364.070648,166.587400
4092,0.655776,0.557722,64.138009,5.569651,19.406519,0.286441,5.134185,41.751688,19.6734,10.213618,2.160973,258.916385,29.670157,2.627395,1.026045,0.541985,309.458343,176.949769
4093,0.824020,0.424761,70.060731,4.900752,17.915470,0.113394,0.268017,51.125728,24.7206,4.420288,0.933583,390.552354,52.783734,3.411842,1.220441,0.620082,445.986053,164.116517
4094,0.714139,0.334924,66.488205,3.919224,21.625679,0.187517,1.236156,58.018170,21.4242,5.356057,1.465975,360.733017,44.419942,5.033340,1.066646,0.585498,321.930873,151.601869


 # INTRAPOLATION CODE BELOW

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load and preprocess data
def load_data():
    mot_a = pd.read_csv("/kaggle/input/motors/Mot_A_rel_2.csv")
    mot_b = pd.read_csv("/kaggle/input/motors/Mot_B_rel_1.csv")
    
    X_a, Y_a = mot_a.iloc[:, :11].values, mot_a.iloc[:, 11:].values
    X_b, Y_b = mot_b.iloc[:, :11].values, mot_b.iloc[:, 11:].values
    
    X_combined = np.vstack([X_a, X_b])
    Y_combined = np.vstack([Y_a, Y_b])
    
    scaler_x, scaler_y = StandardScaler(), StandardScaler()
    X_combined = scaler_x.fit_transform(X_combined)
    Y_combined = scaler_y.fit_transform(Y_combined)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X_combined, Y_combined, test_size=0.2, random_state=42)
    
    return (torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.float32)), \
           (torch.tensor(X_test, dtype=torch.float32), torch.tensor(Y_test, dtype=torch.float32)), \
           scaler_x, scaler_y

# Define MAML Model
class MAMLModel(nn.Module):
    def __init__(self, input_size=11, output_size=7, hidden_size=128):
        super(MAMLModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.fc3(x)

# MAML Meta-Learning Class
class MAML:
    def __init__(self, model, meta_lr=0.001, inner_lr=0.01, inner_steps=1, device="cpu"):
        self.model = model.to(device)
        self.meta_optimizer = optim.Adam(self.model.parameters(), lr=meta_lr)
        self.inner_lr = inner_lr
        self.inner_steps = inner_steps
        self.loss_fn = nn.MSELoss()
        self.device = device

    def inner_update(self, X, y):
        X, y = X.to(self.device), y.to(self.device)
        model_copy = MAMLModel().to(self.device)
        model_copy.load_state_dict(self.model.state_dict())
        optimizer = optim.SGD(model_copy.parameters(), lr=self.inner_lr)
        
        for _ in range(self.inner_steps):
            loss = self.loss_fn(model_copy(X), y)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)  # ✅ Ensure graph is retained
            optimizer.step()
        
        return model_copy

    def meta_train(self, X_train, y_train, epochs=1000):
        X_train, y_train = X_train.to(self.device), y_train.to(self.device)
        
        for epoch in range(epochs):
            model_copy = self.inner_update(X_train, y_train)
            loss = self.loss_fn(self.model(X_train), y_train)  # ✅ Compute loss using meta-model
            
            self.meta_optimizer.zero_grad()
            grads = torch.autograd.grad(loss, self.model.parameters(), create_graph=True, retain_graph=True)
            
            for param, grad in zip(self.model.parameters(), grads):
                if grad is not None:
                    param.grad = grad  # ✅ Correctly setting gradient
            
            self.meta_optimizer.step()
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Meta Loss: {loss.item()}")
    
    def fine_tune(self, X, y, steps=10):
        X, y = X.to(self.device), y.to(self.device)
        model_copy = self.inner_update(X, y)
        optimizer = optim.SGD(model_copy.parameters(), lr=self.inner_lr)
        
        for _ in range(steps):
            loss = self.loss_fn(model_copy(X), y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        return model_copy

# Run MAML Training
device = "cuda" if torch.cuda.is_available() else "cpu"
(X_train, y_train), (X_test, y_test), scaler_x, scaler_y = load_data()

model = MAMLModel()
maml = MAML(model, device=device)

maml.meta_train(X_train, y_train)

adapted_model = maml.fine_tune(X_test, y_test)

y_pred = adapted_model(X_test.to(device)).detach().cpu().numpy()
y_pred = scaler_y.inverse_transform(y_pred)
y_test = scaler_y.inverse_transform(y_test.numpy())

# Rigorous evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Final MSE on 20% test set: {mse}")
print(f"Final RMSE on 20% test set: {rmse}")
print(f"Final MAE on 20% test set: {mae}")
print(f"Final R² Score on 20% test set: {r2}")


Epoch 0, Meta Loss: 1.0210092067718506
Epoch 100, Meta Loss: 0.10090705752372742
Epoch 200, Meta Loss: 0.07084202021360397
Epoch 300, Meta Loss: 0.05194752290844917
Epoch 400, Meta Loss: 0.04164756089448929
Epoch 500, Meta Loss: 0.03286929428577423
Epoch 600, Meta Loss: 0.027026664465665817
Epoch 700, Meta Loss: 0.02323191799223423
Epoch 800, Meta Loss: 0.02055322378873825
Epoch 900, Meta Loss: 0.01929798536002636
Final MSE on 20% test set: 86.39111328125
Final RMSE on 20% test set: 9.294681549072266
Final MAE on 20% test set: 3.704435348510742
Final R² Score on 20% test set: 0.9760610411873577


## EXTRAPOLATION CODE BELOW


# Training on 90% A & 20 % B
# Test on only 80% B



In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load and preprocess data
def load_data():
    mot_a = pd.read_csv("/kaggle/input/motors/Mot_A_rel_2.csv")
    mot_b = pd.read_csv("/kaggle/input/motors/Mot_B_rel_1.csv")
    
    X_a, Y_a = mot_a.iloc[:, :11].values, mot_a.iloc[:, 11:].values
    X_b, Y_b = mot_b.iloc[:, :11].values, mot_b.iloc[:, 11:].values
    
    scaler_x, scaler_y = StandardScaler(), StandardScaler()
    
    X_a = scaler_x.fit_transform(X_a)
    Y_a = scaler_y.fit_transform(Y_a)
    X_b = scaler_x.transform(X_b)
    Y_b = scaler_y.transform(Y_b)
    
    X_train_a, X_test_a, Y_train_a, Y_test_a = train_test_split(X_a, Y_a, test_size=0.1, random_state=42)
    X_train_b, X_test_b, Y_train_b, Y_test_b = train_test_split(X_b, Y_b, test_size=0.8, random_state=42)  # ✅ Hold out 50% of Motor B as unseen Motor C
    
    return {
        "motor_a": (torch.tensor(X_train_a, dtype=torch.float32), torch.tensor(Y_train_a, dtype=torch.float32)),
        "motor_b": (torch.tensor(X_train_b, dtype=torch.float32), torch.tensor(Y_train_b, dtype=torch.float32)),
        "test": (torch.tensor(X_test_b, dtype=torch.float32), torch.tensor(Y_test_b, dtype=torch.float32)),  # ✅ Treat as Motor C (unseen)
        "scalers": (scaler_x, scaler_y)
    }

# Define MAML Model
class MAMLModel(nn.Module):
    def __init__(self, input_size=11, output_size=7, hidden_size=128):
        super(MAMLModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu2(self.fc2(x))
        return self.fc3(x)

# MAML Meta-Learning Class
class MAML:
    def __init__(self, model, meta_lr=0.001, inner_lr=0.01, inner_steps=1, device="cpu"):
        self.model = model.to(device)
        self.meta_optimizer = optim.Adam(self.model.parameters(), lr=meta_lr)
        self.inner_lr = inner_lr
        self.inner_steps = inner_steps
        self.loss_fn = nn.MSELoss()
        self.device = device

    def inner_update(self, X, y):
        X, y = X.to(self.device), y.to(self.device)
        model_copy = MAMLModel().to(self.device)
        model_copy.load_state_dict(self.model.state_dict())
        optimizer = optim.SGD(model_copy.parameters(), lr=self.inner_lr)
        
        for _ in range(self.inner_steps):
            loss = self.loss_fn(model_copy(X), y)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)  # ✅ Ensure graph is retained
            optimizer.step()
        
        return model_copy

    def meta_train(self, tasks, epochs=1000):
        for epoch in range(epochs):
            meta_loss = 0
            for task_name, (X_train, y_train) in tasks.items():
                model_copy = self.inner_update(X_train, y_train)
                loss = self.loss_fn(self.model(X_train.to(self.device)), y_train.to(self.device))
                meta_loss += loss
            
            meta_loss /= len(tasks)  # ✅ Aggregate loss across tasks
            
            self.meta_optimizer.zero_grad()
            grads = torch.autograd.grad(meta_loss, self.model.parameters(), create_graph=True, retain_graph=True)
            
            for param, grad in zip(self.model.parameters(), grads):
                if grad is not None:
                    param.grad = grad  # ✅ Correctly setting gradient
            
            self.meta_optimizer.step()
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Meta Loss: {meta_loss.item()}")
    
    def fine_tune(self, X, y, steps=10):
        X, y = X.to(self.device), y.to(self.device)
        model_copy = self.inner_update(X, y)
        optimizer = optim.SGD(model_copy.parameters(), lr=self.inner_lr)
        
        for _ in range(steps):
            loss = self.loss_fn(model_copy(X), y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        return model_copy

# Run MAML Training
device = "cuda" if torch.cuda.is_available() else "cpu"
data = load_data()

model = MAMLModel()
maml = MAML(model, device=device)

maml.meta_train({"motor_a": data["motor_a"], "motor_b": data["motor_b"]})  # ✅ Train on Motor A & B

adapted_model = maml.fine_tune(*data["test"])  # ✅ Fine-tune on unseen Motor C (held-out part of Motor B)

y_pred = adapted_model(data["test"][0].to(device)).detach().cpu().numpy()
y_pred = data["scalers"][1].inverse_transform(y_pred)  # ✅ Use Motor C's scaler
y_test = data["scalers"][1].inverse_transform(data["test"][1].numpy())

# Rigorous evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Final MSE on simulated Motor C: {mse}")
print(f"Final RMSE on simulated Motor C: {rmse}")
print(f"Final MAE on simulated Motor C: {mae}")
print(f"Final R² Score on simulated Motor C: {r2}")


Epoch 0, Meta Loss: 6.084705829620361
Epoch 100, Meta Loss: 0.27232009172439575
Epoch 200, Meta Loss: 0.16080611944198608
Epoch 300, Meta Loss: 0.11853639781475067
Epoch 400, Meta Loss: 0.09423196315765381
Epoch 500, Meta Loss: 0.0787685215473175
Epoch 600, Meta Loss: 0.06691930443048477
Epoch 700, Meta Loss: 0.05648048594594002
Epoch 800, Meta Loss: 0.04813629388809204
Epoch 900, Meta Loss: 0.041935428977012634
Epoch 1000, Meta Loss: 0.037475451827049255
Epoch 1100, Meta Loss: 0.033876195549964905
Epoch 1200, Meta Loss: 0.03104274719953537
Epoch 1300, Meta Loss: 0.02899795025587082
Epoch 1400, Meta Loss: 0.026858679950237274
Final MSE on simulated Motor C: 143.76060485839844
Final RMSE on simulated Motor C: 11.990020751953125
Final MAE on simulated Motor C: 4.6782073974609375
Final R² Score on simulated Motor C: 0.9471390221383352


# Training on  80% A and  80% B. Testing on rest combined.


In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load and preprocess data
def load_data():
    mot_a = pd.read_csv("/kaggle/input/motors/Mot_A_rel_2.csv")
    mot_b = pd.read_csv("/kaggle/input/motors/Mot_B_rel_1.csv")
    
    X_a, Y_a = mot_a.iloc[:, :11].values, mot_a.iloc[:, 11:].values
    X_b, Y_b = mot_b.iloc[:, :11].values, mot_b.iloc[:, 11:].values
    
    scaler_x, scaler_y = StandardScaler(), StandardScaler()
    
    X_a = scaler_x.fit_transform(X_a)
    Y_a = scaler_y.fit_transform(Y_a)
    X_b = scaler_x.transform(X_b)
    Y_b = scaler_y.transform(Y_b)
    
    X_train_a, X_test_a, Y_train_a, Y_test_a = train_test_split(X_a, Y_a, test_size=0.2, random_state=42)
    X_train_b, X_test_b, Y_train_b, Y_test_b = train_test_split(X_b, Y_b, test_size=0.2, random_state=42)
    
    return {
        "motor_a": (torch.tensor(X_train_a, dtype=torch.float32), torch.tensor(Y_train_a, dtype=torch.float32)),
        "motor_b": (torch.tensor(X_train_b, dtype=torch.float32), torch.tensor(Y_train_b, dtype=torch.float32)),
        "test": (torch.tensor(np.vstack([X_test_a, X_test_b]), dtype=torch.float32), 
                  torch.tensor(np.vstack([Y_test_a, Y_test_b]), dtype=torch.float32)),  # ✅ Test on combined 20% of A & B
        "scalers": (scaler_x, scaler_y)
    }

# Define MAML Model
class MAMLModel(nn.Module):
    def __init__(self, input_size=11, output_size=7, hidden_size=128):
        super(MAMLModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.relu2(self.bn2(self.fc2(x)))
        return self.fc3(x)

# MAML Meta-Learning Class
class MAML:
    def __init__(self, model, meta_lr=0.001, inner_lr=0.01, inner_steps=1, device="cpu"):
        self.model = model.to(device)
        self.meta_optimizer = optim.Adam(self.model.parameters(), lr=meta_lr)
        self.inner_lr = inner_lr
        self.inner_steps = inner_steps
        self.loss_fn = nn.HuberLoss(delta=1.0)
        self.device = device

    def inner_update(self, X, y):
        X, y = X.to(self.device), y.to(self.device)
        model_copy = MAMLModel().to(self.device)
        model_copy.load_state_dict(self.model.state_dict())
        optimizer = optim.Adam(model_copy.parameters(), lr=self.inner_lr)
        
        for _ in range(self.inner_steps):
            loss = self.loss_fn(model_copy(X), y)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)  # ✅ Ensure graph is retained
            optimizer.step()
        
        return model_copy

    def meta_train(self, tasks, epochs=1500):
        for epoch in range(epochs):
            meta_loss = 0
            for task_name, (X_train, y_train) in tasks.items():
                model_copy = self.inner_update(X_train, y_train)
                loss = self.loss_fn(self.model(X_train.to(self.device)), y_train.to(self.device))
                meta_loss += loss
            
            meta_loss /= len(tasks)  # ✅ Aggregate loss across tasks
            
            self.meta_optimizer.zero_grad()
            grads = torch.autograd.grad(meta_loss, self.model.parameters(), create_graph=True, retain_graph=True)
            
            for param, grad in zip(self.model.parameters(), grads):
                if grad is not None:
                    param.grad = grad  # ✅ Correctly setting gradient
            
            self.meta_optimizer.step()
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Meta Loss: {meta_loss.item()}")
    
    def fine_tune(self, X, y, steps=100):
        X, y = X.to(self.device), y.to(self.device)
        model_copy = self.inner_update(X, y)
        optimizer = optim.Adam(model_copy.parameters(), lr=self.inner_lr)
        
        for _ in range(steps):
            loss = self.loss_fn(model_copy(X), y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        return model_copy

# Run MAML Training
device = "cuda" if torch.cuda.is_available() else "cpu"
data = load_data()

model = MAMLModel()
maml = MAML(model, device=device)

maml.meta_train({"motor_a": data["motor_a"], "motor_b": data["motor_b"]})  # ✅ Train only on Motor A

adapted_model = maml.fine_tune(*data["test"])  # ✅ Fine-tune on full Motor B

y_pred = adapted_model(data["test"][0].to(device)).detach().cpu().numpy()
y_pred = data["scalers"][1].inverse_transform(y_pred)  # ✅ Use Motor B's scaler
y_test = data["scalers"][1].inverse_transform(data["test"][1].numpy())

# Rigorous evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Final MSE on combined testMotor B: {mse}")
print(f"Final RMSE on Motor B: {rmse}")
print(f"Final MAE on Motor B: {mae}")
print(f"Final R² Score on Motor B: {r2}")


Epoch 0, Meta Loss: 1.441690444946289
Epoch 100, Meta Loss: 0.7259581089019775
Epoch 200, Meta Loss: 0.5267054438591003
Epoch 300, Meta Loss: 0.20459023118019104
Epoch 400, Meta Loss: 0.08324484527111053
Epoch 500, Meta Loss: 0.049188368022441864
Epoch 600, Meta Loss: 0.038733936846256256
Epoch 700, Meta Loss: 0.03296838328242302
Epoch 800, Meta Loss: 0.029573433101177216
Epoch 900, Meta Loss: 0.026726502925157547
Epoch 1000, Meta Loss: 0.02435028925538063
Epoch 1100, Meta Loss: 0.023025885224342346
Epoch 1200, Meta Loss: 0.0218255203217268
Epoch 1300, Meta Loss: 0.020890012383461
Epoch 1400, Meta Loss: 0.020596351474523544
Final MSE on combined testMotor B: 292.5282897949219
Final RMSE on Motor B: 17.103458404541016
Final MAE on Motor B: 8.247387886047363
Final R² Score on Motor B: 0.8709897311989698


## 